In [14]:
import kagglehub
import os
import shutil

In [15]:
src_path = kagglehub.dataset_download("alessandrasala79/ai-vs-human-generated-dataset")

In [16]:
destination_path = '../data/raw'

In [18]:
destination_path

'../data/raw'

In [17]:
src_path

'/Users/devin/.cache/kagglehub/datasets/alessandrasala79/ai-vs-human-generated-dataset/versions/4'

In [19]:
for file_name in os.listdir(src_path):
    shutil.move(os.path.join(src_path, file_name), os.path.join(destination_path, file_name))